
# 🏠 Real Estate Data Lab — First Analysis (Commented)
한국 아파트 실거래 데이터 기반의 **기초 분석 노트북**  

**이 노트북에서 수행하는 일**
1) CSV 데이터 로드  
2) 기본 전처리(문자→숫자, 결측치 점검 등)  
3) 간단 통계 요약  
4) 월별 가격 추세 분석  
5) 특정 아파트 / 지역별 비교 분석 템플릿  
6) 차트 저장 함수


In [ ]:

# ✅ 필수 라이브러리 임포트
# - pandas: 표 형식 데이터 처리
# - numpy: 수치 연산(필요 시)
# - matplotlib.pyplot: 시각화(라인 그래프 등)
# 주의: 노트북 내 시각화는 matplotlib만 사용한다.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 노트북 내에서 그래프가 보이도록 설정
%matplotlib inline


In [ ]:

# ✅ CSV 데이터 로드
# - file_path 경로만 네 프로젝트에 맞게 수정하면 된다.
# - 기본적으로 'data/apt.csv'를 가정한다.
# - CSV가 다른 인코딩(euc-kr 등)일 경우, encoding 인자를 지정하라.

file_path = "../data/apt.csv"  # 필요에 맞게 변경
# df = pd.read_csv(file_path, encoding="euc-kr")
df = pd.read_csv(file_path)

# 데이터 상위 5행 미리보기
df.head()


In [ ]:

# ✅ 기본 전처리 1: 거래금액을 정수(int)로 변환
# - 실거래 데이터의 '거래금액'이 '12,345'처럼 콤마 포함 문자열인 경우가 많다.
# - 콤마 제거 후 정수형으로 변환한다.
# - 오류가 있으면 에러가 발생하니, 필요 시 errors='coerce'로 결측 처리 가능.

df['거래금액'] = (
    df['거래금액']
    .astype(str)
    .str.replace(",", "", regex=False)
    .str.strip()
    .astype(int)
)


In [ ]:

# ✅ 기본 전처리 2: 날짜 관련(계약년, 계약월, 계약일), 아파트명 정리
# - '계약년월' 컬럼이 'YYYYMM' 형태라고 가정한다.
# - '계약일'은 1~31 사이 값. 자리수를 맞추기 위해 zero-fill(두 자리) 처리한다.
# - '아파트' 컬럼은 문자열로 통일한다.

df['계약년'] = df['계약년월'].astype(str).str[:4]
df['계약월'] = df['계약년월'].astype(str).str[4:6]
df['계약일'] = df['계약일'].astype(str).str.zfill(2)
df['아파트'] = df['아파트'].astype(str).str.strip()


In [ ]:

# ✅ 데이터 구조/기초 통계 확인
# - info(): 컬럼 타입/결측치 현황 확인
# - describe(): 기초 통계량(평균, 표준편차 등) 확인

display(df.info())
display(df.describe())


In [ ]:

# ✅ 전체 가격 개요(평균/최고/최저) 계산
avg_price = df['거래금액'].mean()
max_price = df['거래금액'].max()
min_price = df['거래금액'].min()

print(f"평균 거래금액: {avg_price:,.0f} (만원 기준일 수 있음)")
print(f"최고 거래금액: {max_price:,.0f}")
print(f"최저 거래금액: {min_price:,.0f}")


In [ ]:

# ✅ 가장 자주 거래된 아파트 TOP 10
# - value_counts()로 빈도수 계산
top_apts = df['아파트'].value_counts().head(10)
display(top_apts)


In [ ]:

# ✅ 월 단위(YYYY-MM) 컬럼 생성
# - 월별 집계를 위해 'YYYY-MM' 형식의 문자열을 만든다.
df['월'] = df['계약년'].astype(str) + "-" + df['계약월'].astype(str)
df[['계약년월', '월']].head()


In [ ]:

# ✅ 월별 평균 거래금액 집계
monthly = df.groupby('월', as_index=True)['거래금액'].mean().sort_index()
monthly.head(12)


In [ ]:

# ✅ 월별 평균 거래금액 시각화
# - matplotlib를 사용해 라인 차트를 그린다.
# - x축 라벨이 겹치면 회전(rotation)을 조절한다.

plt.figure(figsize=(12, 4))
plt.plot(monthly.index, monthly.values)
plt.xticks(rotation=45)
plt.title("월별 평균 아파트 매매가")
plt.ylabel("거래금액(만원)")
plt.tight_layout()
plt.show()


In [ ]:

# ✅ 특정 아파트 분석 템플릿
# - 아래 target 값을 원하는 단지명으로 변경하면 해당 단지만 필터링할 수 있다.
# - 예: target = "래미안퍼스티지"
target = "래미안퍼스티지"  # 원하는 아파트명으로 변경

apt_df = df[df['아파트'] == target].copy()

# 월별 평균 가격
apt_monthly = apt_df.groupby('월', as_index=True)['거래금액'].mean().sort_index()

# 미리보기
display(apt_df.head())
display(apt_monthly.head(12))


In [ ]:

# ✅ 특정 아파트 월별 평균 가격 시각화
plt.figure(figsize=(12, 4))
plt.plot(apt_monthly.index, apt_monthly.values)
plt.xticks(rotation=45)
plt.title(f"{target} 월별 평균 매매가")
plt.ylabel("거래금액(만원)")
plt.tight_layout()
plt.show()


In [ ]:

# ✅ 지역(법정동)별 평균 거래금액 TOP 10
# - 지역 단위 비교를 위한 기초 분석
# - 실제 분석에서는 '시/구/동' 등의 계층 컬럼을 함께 사용하면 더 유용하다.
if '법정동' in df.columns:
    location_avg = df.groupby('법정동', as_index=True)['거래금액'].mean().sort_values(ascending=False)
    display(location_avg.head(10))
else:
    print("⚠️ '법정동' 컬럼이 없어 지역별 비교를 생략한다.")


In [ ]:

# ✅ 차트 저장 함수 (선택)
# - charts/ 폴더에 PNG로 저장한다.
# - 디렉토리가 없으면 생성하도록 할 수도 있다.

import os

def save_plot(fig, filename: str, folder: str = "../charts"):
    os.makedirs(folder, exist_ok=True)
    path = os.path.join(folder, f"{filename}.png")
    fig.savefig(path, dpi=200, bbox_inches='tight')
    print(f"저장 완료: {path}")


In [ ]:

# ✅ (예시) 방금 그린 월별 평균 매매가 차트를 저장하고 싶다면 다음처럼 사용한다.
# - 먼저 fig, ax를 생성하여 plot을 그리고 save_plot 호출

fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(monthly.index, monthly.values)
ax.set_title("월별 평균 아파트 매매가")
ax.set_ylabel("거래금액(만원)")
for label in ax.get_xticklabels():
    label.set_rotation(45)
fig.tight_layout()

# 파일 저장
save_plot(fig, "monthly_avg_price")



## ✔ 분석 완료

이 노트북은 Real Estate Data Lab의 **주요 기초 분석 템플릿**이다.  
- CSV → 전처리 → 통계/빈도 → 월별 추세 → 특정 아파트/지역 비교까지 포함했다.  
- 필요에 따라 전세가율, 거래량, 가격지수 결합 분석 등으로 확장할 수 있다.

> 데이터 스키마/컬럼명이 다를 수 있으므로, 필드명만 프로젝트에 맞게 조정해 사용하라.
